In [8]:
from qiskit import Aer, transpile, assemble
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np


In [9]:
def create_eigs(matrix, num_auxiliary, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_auxiliary += 1
        ne_qfts = [QFT(num_auxiliary - 1), QFT(num_auxiliary - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_auxiliary).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_auxiliary,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [10]:
matrix = [[1, -1/3], [-1/3, 1]]
vector = [1, 0]



In [11]:
orig_size = len(vector)
matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrix, vector)
# Initialize eigenvalue finding module
eigs = create_eigs(matrix, 3, 50, False)
num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=vector)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

In [12]:
def fidelity(hhl, ref):
    solution_hhl_normed = hhl / np.linalg.norm(hhl)
    solution_ref_normed = ref / np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed)
    print("Fidelity:\t\t %f" % fidelity)

In [13]:
result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:\t\t", np.round(result['solution'], 5))

result_ref = NumPyLSsolver(matrix, vector).run()
print("Classical Solution:\t", np.round(result_ref['solution'], 5))

print("Probability:\t\t %f" % result['probability_result'])
fidelity(result['solution'], result_ref['solution'])

Traceback (most recent call last):
  Input In [13] in <cell line: 1>
    result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
  File /opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/quantum_algorithm.py:71 in run
    return self._run()
  File /opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/linear_solvers/hhl.py:410 in _run
    self.construct_circuit(measurement=False)
  File /opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/linear_solvers/hhl.py:218 in construct_circuit
    _append_by_qreg(qc, eigs)
  File /opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/linear_solvers/hhl.py:499 in _append_by_qreg
    target._qubit_set.update(element[:])
AttributeError: 'QuantumCircuit' object has no attribute '_qubit_set'

Use %tb to get the full traceback.


In [14]:
%tb

AttributeError: 'QuantumCircuit' object has no attribute '_qubit_set'